# Building a helicopter detector

In [1]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import time
from tqdm import tqdm_notebook
import glob
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread, imsave
from sklearn.model_selection import train_test_split
import numpy as np

#from keras import backend as K
print(tf.__version__)
# create the base pre-trained model
base_model = k.applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False)
#base_model = InceptionV3(weights='imagenet', include_top=False)
base_model.summary()

2.0.0-beta1


/home/alex/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
_______________________________________________________________________________

## 1. Transfer learning

In [2]:
# Add the helicopter class
# We will run a helicopter vs non helicopter inference
x = base_model.output
# Add a global_average_pooling2d layer like in the original model
x = GlobalAveragePooling2D()(x)
# and a sigmoid layer -- helicopter or not helicopter
predictions = Dense(2, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional MobileNetV2 layers
for layer in base_model.layers:
    layer.trainable = False

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [69]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
# train the model on the new data for a few epochs
# Depending on how long it takes might need the TPU
#model.fit_generator(...)
inputPos = np.array([f for f in glob.glob("../0_Database/Processed/*.png")])
inputNeg = np.array([f for f in glob.glob("../0_Database/Negatives/190606001/*.JPG")])
print("Detected {} positive images".format(len(inputPos)))
print("Detected {} negative images".format(len(inputNeg)))
X = []
Y = []
for img in tqdm_notebook(inputPos):
    X.append(imread(img)[:, :, :3])
    Y.append([1, 0])

for index, img in enumerate(tqdm_notebook(inputNeg)):
    # Images are random so let's take a random crop for all of them
    X.append(imread(img)[2000:2224, 2000:2224, :3])
    Y.append([0, 1])
    if index > 20:
        break



Detected 174 positive images
Detected 105 negative images


In [71]:
# Transform the list of np array to a 4D matrix
X = np.c_[X]
Y = np.array(Y)
print("X is {}".format(X.shape))
print("Y is {}".format(Y.shape))

# Shuffle the array
trainSplit = 0.8
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

#trainIndexes = np.random.permutation(round(trainSplit*len(X)))
#X_train = X[trainIndexes]
#Y_train = Y[trainIndexes]
print("Train set has {} images".format(len(X_train)))

X is (196, 224, 224, 3)
Y is (196, 2)
Train set has 156 images


In [72]:
nbEpoch = 5
model.fit(X_train, Y_train, validation_split = 0.1, epochs=nbEpoch)

Train on 140 samples, validate on 16 samples
Epoch 1/5
140/140 [==============================] - 6s 43ms/sample - loss: 0.4309 - accuracy: 0.8786 - val_loss: 0.7264 - val_accuracy: 0.8125
Epoch 2/5
140/140 [==============================] - 4s 27ms/sample - loss: 0.2905 - accuracy: 0.8821 - val_loss: 0.5935 - val_accuracy: 0.8125
Epoch 3/5
140/140 [==============================] - 6s 40ms/sample - loss: 0.2324 - accuracy: 0.9143 - val_loss: 0.5142 - val_accuracy: 0.8125
Epoch 4/5
140/140 [==============================] - 6s 41ms/sample - loss: 0.1452 - accuracy: 0.9679 - val_loss: 0.4676 - val_accuracy: 0.8125
Epoch 5/5
140/140 [==============================] - 6s 40ms/sample - loss: 0.1269 - accuracy: 0.9679 - val_loss: 0.4777 - val_accuracy: 0.8125


In [75]:
modelPred = np.round(model.predict(X_test))
# Test accuracy
accuracy = 1-np.sum(np.abs(Y_test-modelPred), axis=0)[0]/len(X_test)
print("Model accuracy: {}".format(accuracy))

Model accuracy: 0.95


In [ ]:
# Is the alpha component really that interesting? No
index = np.random.randint(len(inputPos))
image = imread(inputPos[index])
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(image[:, :, :3])
ax[0].set_title("Without alpha")
ax[0].axis("off")
ax[1].imshow(image)
ax[1].set_title("With alpha")
ax[1].axis("off")

## 2. Benchmark result

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)